In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/data mining

/content/drive/MyDrive/data mining


In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
file_path='data/train_2.txt'

In [ ]:
def loadInputFile(file_path):
    trainingset = list()  # store trainingset [content,content,...]
    position = list()  # store position [article_id, start_pos, end_pos, entity_text, entity_type, ...]
    mentions = dict()  # store mentions[mention] = Type
    with open(file_path, 'r', encoding='utf8') as f:
        file_text=f.read().encode('utf-8').decode('utf-8-sig')
    datas=file_text.split('\n\n--------------------\n\n')[:-1]
    for data in datas:
        data=data.split('\n')
        content=data[0]
        trainingset.append(content)
        annotations=data[1:]
        for annot in annotations[1:]:
            annot=annot.split('\t') #annot= article_id, start_pos, end_pos, entity_text, entity_type
            position.extend(annot)
            mentions[annot[3]]=annot[4]
    
    return trainingset, position, mentions

In [ ]:
def CRFFormatData(trainingset, position, path):
    if (os.path.isfile(path)):
        os.remove(path)
    outputfile = open(path, 'a', encoding= 'utf-8')

    # output file lines
    count = 0 # annotation counts in each content
    tagged = list()
    for article_id in range(len(trainingset)):
        trainingset_split = list(trainingset[article_id])
        while '' or ' ' in trainingset_split:
            if '' in trainingset_split:
                trainingset_split.remove('')
            else:
                trainingset_split.remove(' ')
        start_tmp = 0
        for position_idx in range(0,len(position),5):
            if int(position[position_idx]) == article_id:
                count += 1
                if count == 1:
                    start_pos = int(position[position_idx+1])
                    end_pos = int(position[position_idx+2])
                    entity_type=position[position_idx+4]
                    if start_pos == 0:
                        token = list(trainingset[article_id][start_pos:end_pos])
                        whole_token = trainingset[article_id][start_pos:end_pos]
                        for token_idx in range(len(token)):
                            if len(token[token_idx].replace(' ','')) == 0:
                                continue
                            # BIO states
                            if token_idx == 0:
                                label = 'B-'+entity_type
                            else:
                                label = 'I-'+entity_type
                            
                            output_str = token[token_idx] + ' ' + label + '\n'
                            outputfile.write(output_str)

                    else:
                        token = list(trainingset[article_id][0:start_pos])
                        whole_token = trainingset[article_id][0:start_pos]
                        for token_idx in range(len(token)):
                            if len(token[token_idx].replace(' ','')) == 0:
                                continue
                            
                            output_str = token[token_idx] + ' ' + 'O' + '\n'
                            outputfile.write(output_str)

                        token = list(trainingset[article_id][start_pos:end_pos])
                        whole_token = trainingset[article_id][start_pos:end_pos]
                        for token_idx in range(len(token)):
                            if len(token[token_idx].replace(' ','')) == 0:
                                continue
                            # BIO states
                            if token[0] == '':
                                if token_idx == 1:
                                    label = 'B-'+entity_type
                                else:
                                    label = 'I-'+entity_type
                            else:
                                if token_idx == 0:
                                    label = 'B-'+entity_type
                                else:
                                    label = 'I-'+entity_type

                            output_str = token[token_idx] + ' ' + label + '\n'
                            outputfile.write(output_str)

                    start_tmp = end_pos
                else:
                    start_pos = int(position[position_idx+1])
                    end_pos = int(position[position_idx+2])
                    entity_type=position[position_idx+4]
                    if start_pos<start_tmp:
                        continue
                    else:
                        token = list(trainingset[article_id][start_tmp:start_pos])
                        whole_token = trainingset[article_id][start_tmp:start_pos]
                        for token_idx in range(len(token)):
                            if len(token[token_idx].replace(' ','')) == 0:
                                continue
                            output_str = token[token_idx] + ' ' + 'O' + '\n'
                            outputfile.write(output_str)

                    token = list(trainingset[article_id][start_pos:end_pos])
                    whole_token = trainingset[article_id][start_pos:end_pos]
                    for token_idx in range(len(token)):
                        if len(token[token_idx].replace(' ','')) == 0:
                            continue
                        # BIO states
                        if token[0] == '':
                            if token_idx == 1:
                                label = 'B-'+entity_type
                            else:
                                label = 'I-'+entity_type
                        else:
                            if token_idx == 0:
                                label = 'B-'+entity_type
                            else:
                                label = 'I-'+entity_type
                        
                        output_str = token[token_idx] + ' ' + label + '\n'
                        outputfile.write(output_str)
                    start_tmp = end_pos

        token = list(trainingset[article_id][start_tmp:])
        whole_token = trainingset[article_id][start_tmp:]
        for token_idx in range(len(token)):
            if len(token[token_idx].replace(' ','')) == 0:
                continue

            
            output_str = token[token_idx] + ' ' + 'O' + '\n'
            outputfile.write(output_str)

        count = 0
    
        output_str = '\n'
        outputfile.write(output_str)
        ID = trainingset[article_id]

        if article_id%10 == 0:
            print('Total complete articles:', article_id)

    # close output file
    outputfile.close()

In [ ]:
trainingset, position, mentions=loadInputFile(file_path)
data_path='data/sample.data'
CRFFormatData(trainingset, position, data_path)

Total complete articles: 0
Total complete articles: 10
Total complete articles: 20
Total complete articles: 30
Total complete articles: 40
Total complete articles: 50
Total complete articles: 60
Total complete articles: 70
Total complete articles: 80
Total complete articles: 90
Total complete articles: 100
Total complete articles: 110
Total complete articles: 120
Total complete articles: 130
Total complete articles: 140
Total complete articles: 150
Total complete articles: 160
Total complete articles: 170
Total complete articles: 180
Total complete articles: 190


In [ ]:
!pip install sklearn_crfsuite

In [ ]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn_crfsuite.metrics import flat_classification_report

In [ ]:
# load `train.data` and separate into a list of labeled data of each text
# return:
#   data_list: a list of lists of tuples, storing tokens and labels (wrapped in tuple) of each text in `train.data`
#   traindata_list: a list of lists, storing training data_list splitted from data_list
#   testdata_list: a list of lists, storing testing data_list splitted from data_list
from sklearn.model_selection import train_test_split
def Dataset(data_path):
    with open(data_path, 'r', encoding='utf-8') as f:
        data=f.readlines()#.encode('utf-8').decode('utf-8-sig')
    data_list, data_list_tmp = list(), list()
    article_id_list=list()
    idx=0
    for row in data:
        data_tuple = tuple()
        if row == '\n':
            article_id_list.append(idx)
            idx+=1
            data_list.append(data_list_tmp)
            data_list_tmp = []
        else:
            row = row.strip('\n').split(' ')
            data_tuple = (row[0], row[1])
            data_list_tmp.append(data_tuple)
    if len(data_list_tmp) != 0:
        data_list.append(data_list_tmp)
    
    # here we random split data into training dataset and testing dataset
    # but you should take `development data` or `test data` as testing data
    # At that time, you could just delete this line, 
    # and generate data_list of `train data` and data_list of `development/test data` by this function
    traindata_list, testdata_list, traindata_article_id_list, testdata_article_id_list=train_test_split(data_list,
                                                                                                    article_id_list,
                                                                                                    test_size=0.2,
                                                                                                    random_state=42)
    
    return data_list, traindata_list, testdata_list, traindata_article_id_list, testdata_article_id_list 

In [ ]:
data_list, traindata_list, testdata_list, traindata_article_id_list, testdata_article_id_list = Dataset(data_path)

In [ ]:
totaldata_list = traindata_list + testdata_list

In [ ]:
def strQ2B(ustring):
    """把字串全形轉半形"""
    ss = []
    for s in ustring:
        rstring = ""
        for uchar in s:
            inside_code = ord(uchar)
            if inside_code == 12288:  # 全形空格直接轉換
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全形字元（除空格）根據關係轉化
                inside_code -= 65248
            rstring += chr(inside_code)
        ss.append(rstring)
    return ''.join(ss)

In [ ]:
#最大句號長度
def max_dot_split(traindata_list,max_seqence=512):
    totalSeq = []
    totalValue = []
    max_seqence = max_seqence
    for article in traindata_list:
        #print(article)
        seqText = []
        seqValue = []
        batch_seq_text = []
        batch_seq_value = []
        batch_length  = 0
        seq_length = 0

        for index in range(len(article)):

            word = article[index][0]
            word = strQ2B(word)
            value = article[index][1]
            #seqText += word+'\x02'
            #seqValue += value+'\x02'
            seqText.append(word)
            seqValue.append(value)
            seq_length += 1

            if(word == '。'):
                batch_length += seq_length
                if(batch_length <= max_seqence):
                    batch_seq_text += seqText
                    batch_seq_value += seqValue
                    seq_length = 0

                if(batch_length > max_seqence):
                    #print('a batch', len(batch_seq_text))
                    totalSeq.append(batch_seq_text)
                    totalValue.append(batch_seq_value)
                    if(index == len(article)-1):
                        totalSeq.append(seqText)
                        totalValue.append(seqValue)
                        #print('a batch', len(seqText))
                    batch_length = seq_length
                    batch_seq_text = seqText
                    batch_seq_value = seqValue
                    seq_length = 0
                
                seqText = []
                seqValue = []
    return totalSeq,totalValue

In [ ]:
train_x,train_y = max_dot_split(traindata_list,128)
valid_x, valid_y = max_dot_split(testdata_list,128)

In [ ]:
!pip install "kashgari>=1.1,<2.0"
!pip install "tensorflow-gpu>=1.14,<2.0"


In [ ]:
# from kashgari.corpus import ChineseDailyNerCorpus

# train_x, train_y = ChineseDailyNerCorpus.load_data('train')
# valid_x, valid_y = ChineseDailyNerCorpus.load_data('validate')
# test_x, test_y  = ChineseDailyNerCorpus.load_data('test')

In [ ]:
# !unzip data/data65800/chinese_wwm_ext_L-12_H-768_A-12.zip -d bert 
# !unzip data/data65800/roeberta_zh_L-24_H-1024_A-16.zip -d roberta 

In [ ]:
import kashgari
from kashgari.embeddings import BERTEmbedding

bert_embed = BERTEmbedding('embeddings/chinese_wwm_ext_L-12_H-768_A-12',
                           task=kashgari.LABELING,
                           sequence_length=128)

/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:22: UserWarning: bert4keras.bert has been renamed as bert4keras.models.
  warnings.warn('bert4keras.bert has been renamed as bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:23: UserWarning: please use bert4keras.models.
  warnings.warn('please use bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:44: UserWarning: bert4keras.tokenizer has been renamed as bert4keras.tokenizers.
  warnings.warn('bert4keras.tokenizer has been renamed as bert4keras.tokenizers.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:45: UserWarning: please use bert4keras.tokenizers.
  warnings.warn('please use bert4keras.tokenizers.')


In [ ]:
# import os
# import kashgari
# from kashgari.embeddings import TransformerEmbedding
# from kashgari.tokenizer import BertTokenizer 
# # Setup paths
# model_folder = ''
# checkpoint_path = os.path.join(model_folder, 'roberta_zh_large_model.ckpt.data-00000-of-00001')
# config_path = os.path.join(model_folder, 'bert_config.json')
# vocab_path = os.path.join(model_folder, 'vocab.txt')

# tokenizer = BertTokenizer.load_from_vocab_file(vocab_path)
# embed = TransformerEmbedding(vocab_path, config_path, checkpoint_path,
#                              bert_type='roberta')

In [ ]:
from kashgari.tasks.labeling import BiLSTM_CRF_Model

# 还可以选择 `CNN_LSTM_Model`, `BiLSTM_Model`, `BiGRU_Model` 或 `BiGRU_CRF_Model`

model = BiLSTM_CRF_Model(bert_embed)
model.fit(train_x,
          train_y,
          x_validate=valid_x,
          y_validate=valid_y,
          epochs=20,
          batch_size=8)


/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:22: UserWarning: bert4keras.bert has been renamed as bert4keras.models.
  warnings.warn('bert4keras.bert has been renamed as bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:23: UserWarning: please use bert4keras.models.
  warnings.warn('please use bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:44: UserWarning: bert4keras.tokenizer has been renamed as bert4keras.tokenizers.
  warnings.warn('bert4keras.tokenizer has been renamed as bert4keras.tokenizers.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:45: UserWarning: please use bert4keras.tokenizers.
  warnings.warn('please use bert4keras.tokenizers.')


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 128)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 128, 768), ( 16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 128, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [ ]:
model.save('model_save')

In [ ]:
def get_format_article_result(atricle_id,total_label_tags, total_sentence_str):
    article_result = []

    entiy = ''
    start_position = 0
    end_position = 0
    current_length = 0

    for index in range(len(total_label_tags)):
        label_tag = total_label_tags[index]
        #print('index:', index)
        # if(entiy != ''):
        #     print(index,current_length,entiy)
        if(label_tag.startswith('B-')):
            #if it is a new B, record last entiy
            if(current_length>=2):
                end_position = index
                article_result.append([atricle_id,start_position,end_position,entiy,entity_type])
                #print('record:',[atricle_id,start_position,end_position,entiy,entity_type])
                #record_entiy()

            entiy = ''
            start_position = 0
            end_position = 0
            current_length = 0

            #find the entiy at B-
            #if(current_length == 0):
            entiy += total_sentence_str[index]
            start_position = index
            #print('start position: ', start_position)
            entity_type = label_tag.split("-")[1]
            current_length += 1
                
        
        if(label_tag.startswith('I-')):
            if(index>=1):
                # if the last label is B-
                if(current_length >= 1):
                    if(total_label_tags[index-1].split("-")[1] == label_tag.split("-")[1]):
                        entiy += total_sentence_str[index]
                        current_length += 1 
                    if(total_label_tags[index-1].split("-")[1] != label_tag.split("-")[1]):
                        if(current_length>=2):
                            end_position = index
                            article_result.append([atricle_id,start_position,end_position,entiy,entity_type])
                        entiy = ''
                        start_position = 0
                        end_position = 0
                        current_length = 0
                # if((total_label_tags[index-1].startswith('B-') and total_label_tags[index-1].split("-")[1] != label_tag.split("-")[1]) or (total_label_tags[index-1].startswith('I-') and total_label_tags[index-1].split("-")[1] != label_tag.split("-")[1])):
                #     #print('detect')
                #     # if the label type is not same as the last one
                #     #if(total_label_tags[index-1].split("-")[1] != label_tag.split("-")[1]):
                #     # reinint parm
                #     entiy = ''
                #     start_position = 0
                #     end_position = 0
                #     current_length = 0
                    
                #     # find the entiy by I-
                #     entiy += total_sentence_str[index]
                #     start_position = index
                #     entity_type = label_tag.split("-")[1]
                #     current_length += 1
                # if the last label is not B-, continue the count
                #if(total_label_tags[index-1].startswith('I-') and total_label_tags[index-1].split("-")[1] != label_tag.split("-")[1])
                #else: 

        if(label_tag == 'O'):
            #print('meet O')
            # when meet O only record when the length more than one. ignore the 'only one'
            if(current_length>=2):
                end_position = index
                article_result.append([atricle_id,start_position,end_position,entiy,entity_type])
                #print('record:',[atricle_id,start_position,end_position,entiy,entity_type])
            #record_entiy()

            entiy = ''
            start_position = 0
            end_position = 0
            current_length = 0
            #reinit_parm()
    return article_result
    

In [ ]:
from pandas import DataFrame
import time  # 引入time模块
 
def get_total_result(total_articles,model):
    total_result = []
    for article_index in range(len(total_articles)):
      total_word_list = []
      for sentence in test_data[article_index]:
        total_word_list += sentence
      total_label_tags = []
      for predict in model.predict(test_data[article_index]):
        total_label_tags += predict
      #print(total_word_list)
      #print(total_label_tags)
      total_result += get_format_article_result(articleids[article_index],total_label_tags,total_word_list)
      
      #total_result += get_format_article_result(articleids[article_index],total_label_tags,total_word_list)
    
    result_df = DataFrame (total_result,columns=['article_id','start_position','end_position','entity_text','entity_type'])
    #result_df['entity_text'] = result_df['entity_text'].apply(lambda x: s2t.convert(x))
    #result_df.to_csv(checkpoint_dir_path+'/'+checkpoint_dir_path+'.tsv', sep = '\t',index=False)
    return result_df


In [ ]:
result_df = get_total_result(test_data,model)

NameError: ignored

In [ ]:
result_df['entity_type'].value_counts()

In [ ]:
!pwd

In [ ]:
result_df.to_csv('result.tsv',sep='\t',index=False)